# Anomaly Detection

In this notebook, you will get familiar with Anomaly Detection and Active Learning. Let us start by importing the necessary packages. While solving the exercises, feel free to import additional functionality or packages if required.

In [3]:
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Functions needed in this notebook

Run the following cell **WITHOUT** changing anything.

In [4]:
def plot_decision_boundary(trained_classifier, X, y=None):
    # plot variables
    margin_size = 0.1
    steps=100
    
    # ranges
    x_min, x_max = X[:, 0].min(), X[:, 0].max()
    y_min, y_max = X[:, 1].min(), X[:, 1].max()
    x_range = abs(x_max - x_min)
    y_range = abs(y_max - y_min)
    
    xmin, xmax = x_min - margin_size * x_range, x_max + margin_size * x_range
    ymin, ymax = y_min - margin_size * y_range, y_max + margin_size * y_range
    
    # make the meshgrid based on the data 
    xx, yy = np.meshgrid(
        np.linspace(xmin, xmax, int(steps)),
        np.linspace(ymin, ymax, int(steps)))
    X_mesh = np.c_[xx.ravel(), yy.ravel()]

    # predict for the mesh
    Z = trained_classifier.predict_proba(X_mesh)[:, 1]
    Z = Z.reshape(xx.shape)
    
    # plot the figure
    plt.figure(figsize=(13, 10))

    # plot the contour
    plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, vmin=0.0, vmax=1.0, alpha=0.8)
    plt.colorbar()

    # plot the points
    if y is not None:
        for l in [0, -1, 1]:
            ixl = np.where(y_pred == l)[0]
            plt.scatter(X[ixl, 0], X[ixl, 1], color=plot_colors[l], marker=plot_markers[l])
    else:
        plt.scatter(X[:, 0], X[:, 1], edgecolors="k")
    plt.grid(alpha=0.25)
    plt.title("Decision boundary: red = more anomalous, blue = more normal")
    plt.show()

## Data: Artificial Dataset

For this session we generate our own dataset. Because the goal is to plot the data along with the next steps, the dataset will have only 2 features. In Anomaly Detection the majority of the data is considered as normal, while anomalies represent a small percentage of the total amount. Here, we will set the proportion of normal examples to 95% (class prior). By initializing the random seed of the *numpy* package, we make sure that everyone generates the same dataset.

In [5]:
np.random.seed(1)
class_prior = 0.95

# X
n1 = 200
n2 = 400
n3 = 500
n4 = 50
nn = n1 + n2 + n3 + n4
na = int(nn * (1.0 - class_prior))
X = np.vstack((
    np.random.uniform(low=[2, 12], high=[8, 18], size=(n1, 2)),
    np.random.multivariate_normal([7, 5], [[0.2, 0], [0, 0.2]], n2),
    np.random.multivariate_normal([10, 11], [[0.3, 0.4], [0.4, 3]], n3),
    np.random.multivariate_normal([12, 8], [[0.05, 0], [0, 0.05]], n4),
    np.random.uniform(low=[0, 0], high=[15, 20], size=(na, 2)),
))

# y: when the label of an example = 0, it means we do not know its label
Y = np.zeros(len(X), dtype=int)

# let's assume we know 50 normal examples... (label = -1)
rand_ix = np.random.choice(nn, 50, replace=False)
Y[rand_ix] = -1

# ... and 5 anomalous examples (label = 1)
Y[-5:] = 1

# colors and symbols
PLOT_COLORS = {1: "tab:red", -1: "tab:green", 0: "tab:blue"}
PLOT_MARKERS = {1: "^", -1: "*", 0: "o"}

**EXERCISE:** Plot the data in a scatter plot to visually check its structure. You can set the color and the marker of the examples (points) using the given dictionaries `PLOT_COLORS` and `PLOT_MARKERS` 

In [6]:
# SOLUTION






**QUESTION 1:** Can you intuitively identify the points that look like outliers/anomalies when looking at the scatter plot?

In [7]:
# ANSWER

## Anomaly detection: k-nearest neighbour outlier detection (kNN)

First, let's import some functionality from the `sklearn` package. This package is widely used within the machine learning community.

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

Anomaly detection is usually tackled as an unsupervised learning problem for two reasons.
1. Labels are difficult to obtain in real-world anomaly detection problems. This is because anomalies do not happen a lot so you have to wait a long time to label an anomaly. When an anomaly happens, it usually corresponds to a costly problems in the real-world. For instance, a person committing credit card fraud, a machine breaking, or a leak in the water supply system of a building.
2. Supervised learners are not great at learning good decision boundaries when there is a large class imbalance (the number of normal examples >> the number of anomalous examples) and only a handful examples to learn from.

To illustrate this phenomenon, let's try to fit a supervised learning model to the data using the available labeled examples. Then, we can visually inspect how this model classifies the whole dataset and what it's decision boundary looks like.

**EXERCISE:** Fit a supervised `DecisionTreeClassifier` on the labeled subset of `X` (remember the labels are stored in `Y`). Then, use the trained classifier to predict the label of every example in `X` and plot the resulting classification. Finally, plot the decision boundary of this classifier for the dataset using the `plot_decision_boundary` function. Hint: you can read about how to apply a classifier in the documentation of the `sklearn` package [https://scikit-learn.org/stable/modules/tree.html#classification]

*The decision boundary plot indicates how the classifier would classify each region of the plot, indicated using the colorscale. It allows us to visually inspect whether the classifier learned a useful separation between the 2 classes.*

In [9]:
# SOLUTION






**QUESTION 2:** In your opinion, did the classifier learn a useful decision boundary? Does this correspond to your intuition from QUESTION 1?

In [10]:
# ANSWER

To overcome the absence of labels, anomaly detection models exploit underlying intuitions about the anomalies and assign a real-valued score to each example that represent its degree of anomalousness (the higher the more anomalous). The model we will use in this session is called *k-nearest neighbors anomaly detection (kNN)* and it assigns to each example the distance to its k nearest neightbors as an anomaly score.

Because the implementation of the model is straightforward, we will use the implemented version in `PyOD` [https://pyod.readthedocs.io/en/latest/index.html]. `PyOD` is a good library for anomaly detection containing many state-of-the-art methods.

First, we need to install the library and import the model that we want to use.

In [11]:
!pip install pyod
from pyod.models.knn import KNN

**QUESTION 3:** What is the key insight / underlying intuition behind `kNN` outlier detection? Why do greater anomaly scores correspond to more anomalous examples? Provide an answer a-priori, i.e. without running the next code.

In [12]:
# ANSWER

**EXERCISE:** Let's now experimentally see how `kNN` outlier detection works. Fit the model to the full dataset `X`, compute the anomaly score for each example in `X` and plot the resulting scores. Finally, plot the decision boundary of this classifier for the dataset using the `plot_decision_boundary` function. Hint: because `kNN` is an *unsupervised* method, you don't need to use `Y`. You can read about how to apply `kNN` in the documentation of the `PyOD` package [https://pyod.readthedocs.io/en/latest/pyod.models.html#module-pyod.models.knn]. To plot the anomaly scores, you can use the `cmap` argument of the plot function in `matplotlib`.

In [13]:
# SOLUTION






**QUESTION 4:** Can you visually identify the examples with the largest anomaly scores? How does this correspond to your intuition from QUESTION 1? How does the decision boundary compare to the one learned in QUESTION 2?

In [14]:
# ANSWER

Let's now move on to discrete predictions. To do inference, anomaly detection models set a threshold on the anomaly scores using to the contamination factor. Examples with an anomaly score > the threshold are labeled *anomaly*, examples with an anomaly score < threshold are labeled *normal*.

**EXERCISE:** Predict the discrete classes of the examples using `kNN` and starting from the anomaly scores. Plot the results. You can use the `PLOT_MARKERS` and `PLOT_COLORS` objects to plot different colors for the examples, or you can use your own imagination.

In [15]:
# SOLUTION






**QUESTION 5:** Do you agree with this classification? Are there points you would label differently looking at the plot?

In [16]:
# ANSWER

## Anomaly detection: Isolation Forest

In [17]:
from pyod.models.iforest import IForest

There are many unsupervised anomaly detectors, each with their own strenghts and weaknesses. A very popular detector is the *Isolation Forest* or *IForest* detector. The intuition behind this detector is as simple: anomalies are easier to isolate from the bulk of examples than the normals. The algorithm computes an *isolation score* for each example in the dataset. `IForest` works as follows:
1. It constructs an ensemble of trees. Each tree splits the data multiple times using random axis-parallel splits. This means that in each node of the tree, the algorithm picks a random feature and a random split value for that feature (based on the range of the remaining examples in the node). The data are split into to groups and the process repeats itself until each node contains only a single example.
2. For each example, it computes the path length between the root node and the leaf node of that example. In other words, it counts the number of splits needed to isolate the example.
3. It computes the average path length of the example across the ensemble. This is the final anomaly score for that example.

**EXERCISE:** Apply the `IForest` model to the dataset `X`. Then, plot the resulting decision boundary using the `plot_decision_boundary` function. You can also plot the anomaly scores if you want.

In [18]:
# SOLUTION






**QUESTION 6:** How does `IForest` differ from `KNN`? Can you explain some of the differences knowing the internal procedures of both algorithms?

In [19]:
# ANSWER

**EXERCISE:** Try tuning the hyperparameters of `IForest` and `KNN` to get better results. Reason about how these hyperparameters affect the resulting classification.

In [24]:
# SOLUTION






## Anomaly detection biases

To illustrate the biases of different anomaly detectors, we will slightly restructure the dataset we have been working with. We reduce the size of the upper-left cluster and the small rightmost cluster. We also increase the size of the middle 2 clusters. Finally, we drop the labels.

In [20]:
# X
n1 = 100 # <-- 200
n2 = 500 # <-- 400
n3 = 600 # <-- 500
n4 = 20 # <-- 50
nn = n1 + n2 + n3 + n4
na = int(nn * (1.0 - class_prior))
X = np.vstack((
    np.random.uniform(low=[2, 12], high=[8, 18], size=(n1, 2)),
    np.random.multivariate_normal([7, 5], [[0.2, 0], [0, 0.2]], n2),
    np.random.multivariate_normal([10, 11], [[0.3, 0.4], [0.4, 3]], n3),
    np.random.multivariate_normal([12, 8], [[0.05, 0], [0, 0.05]], n4),
    np.random.uniform(low=[0, 0], high=[15, 20], size=(na, 2)),
))

# ... all labels are unknown
Y = np.zeros(len(X), dtype=int)

**EXERCISE:** Plot the updated dataset.

In [21]:
# SOLUTION






We can still clearly see 4 clusters in the dataset. Ideally, the anomaly detector can also discover this structure and find anomalies relative to the clusters. Let's put this theory to the test.

**EXERCISE:** Fit the `KNN` and `IForest` models to the updated dataset `X`. Plot for both algorithms the decision boundaries.

In [22]:
# SOLUTION






**QUESTION 7:** How do `KNN` and `IForest` deal with the change in data density? Did you expect these changes? Why/why not? Do you think that the dataset has fundamentally changed and that the changes in decision boundaries properly reflect these changes?

In [23]:
# ANSWER

It turns out that `IForest` has trouble discovering the relevant structure in the dataset. That is because the splits in the nodes are heavily skewed by the data density, making it difficult for smaller clusters to be identified as normal. However, we can fix this! One solution would be to explicitly extract the clusters from the data and apply `IForest` to each cluster separately. This forces the algorithm to treat each cluster as a separate "context" and only look for anomalies within a certain context.

Let's construct this *augmented* dataset by adding a new feature that specifies the cluster to which each example belongs, i.e., the context (anomalies are assigned to random contexts). Run the following code to construct the new dataset:

In [25]:
# the cluster
X_cluster = np.zeros(len(X), dtype=float)
X_cluster[n1:n1+n2] = 1.0
X_cluster[n1+n2:n1+n2+n3] = 2.0
X_cluster[n1+n2+n3:n1+n2+n3+n4] = 3.0
X_cluster[n1+n2+n3+n4:] = np.random.choice([0.0, 1.0, 2.0, 3.0], na)

**ADVANCED EXERCISE:** Train a separate `IForest` model for each cluster, combine the predictions of the different models. Plot the result.

In [27]:
# SOLUTION






**QUESTION 8:** Does the resulting classification look beter than in QUESTION 7? Do you think we solved the problem now? 

In [28]:
# ANSWER